## Objective

plot US campaigns that have been completed and are currently live (color coded by success, fail, live). 
Generate pop-up with:
 - project name, category
 - goal amount
 - number of backers
 - url 
 

In [4]:
#!pip install geopandas

In [5]:
#!pip install folium

In [59]:
import geopandas as gp
import folium
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import glob
import functools

src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

pd.set_option('display.max_columns', None)
%matplotlib inline 

In [7]:
map_data = pd.read_pickle('../../data/02_intermediate/kickUS_map.pkl')

In [70]:
map_data.head()

,backers_count,blurb,converted_pledged_amount,country,created_at,goal,id,launched_at,name,pledged,source_url,spotlight,staff_pick,state,state_changed_at,usd_pledged,sub_category,overall_category,city,country_loc,state_id,creator_name,lat,lng
0,1,The story of a young man and the girl who brok...,1.0,US,2019-07-18 00:54:35,1500.0,654869661,2019-07-18 01:43:29,Josiah Gennell: Creating First EP,1.0,https://www.kickstarter.com/discover/categorie...,False,False,live,2019-07-18 01:43:29,1.0,Indie Rock,music/indie rock,Methuen,US,MA,Josiah Gennell,42.7340,-71.1889
1,1,CincyPet Magazine is the official lifestyle pe...,1.0,US,2019-07-17 22:27:08,4000.0,1747360837,2019-07-18 02:55:13,CincyPet Magazine,1.0,https://www.kickstarter.com/discover/categorie...,False,False,live,2019-07-18 02:55:13,1.0,Periodicals,publishing/periodicals,Cincinnati,US,OH,Susannah Maynard,39.1412,-84.5059
2,4,I'll be creating an enamel pin of Eleven from ...,76.0,US,2019-07-17 22:04:14,370.0,592255111,2019-07-17 22:50:05,Bad-ass Women Enamel Pins,76.0,https://www.kickstarter.com/discover/categorie...,False,False,live,2019-07-17 22:50:06,76.0,Art,art,Dallas,US,TX,Joelle Nagy,32.7937,-96.7662
3,1,Creating packages for Girl Scout crafting acti...,1.0,US,2019-07-17 21:35:12,10000.0,1561837546,2019-07-17 22:24:13,Creative Craft Kits,1.0,https://www.kickstarter.com/discover/categorie...,False,False,live,2019-07-17 22:24:13,1.0,Crafts,crafts,La Mesa,US,CA,Decca,32.7703,-117.0204
4,0,The film follows 4 frustrated campaigns as the...,0.0,US,2019-07-17 19:45:17,2000.0,255952264,2019-07-18 03:23:01,A Period Piece DVD Funding,0.0,https://www.kickstarter.com/discover/categorie...,False,False,live,2019-07-18 03:23:02,0.0,Drama,film & video/drama,Nashville,US,TN,Anthony Stephen Hamilton,36.1714,-86.7844


99353 NaN values in lat/lng columns. Going to proceed to practice with what I have now

`map_data.lat.isna().sum()`

Need to generate list of popups on subset of data that actually has lat/lng data. 

In [14]:
map_data.lat.mean()

38.07277354688543

In [19]:
df = map_data.loc[(map_data.lat.isna() == False)]

In [62]:
locations = list(zip(df.lat, df.lng))

In [71]:
popup_content = list(zip(df.name, df.backers_count, df.goal))

In [83]:
popups = ['<center> {} <br>  <b>Goal:</b> ${}  <br> <b>BackerCount: </b> {}  </center>'.format(name, goal, count) for (name, count, goal) in popup_content]

len(popups)

123785

In [27]:
from folium.plugins import MarkerCluster

### Failed Attempts at Maps

In [58]:
# m = folium.Map(location=[map_data.lat.mean(), map_data.lng.mean()], zoom_start=5)

# locations = list(zip(df.lat, df.lng))

# icon_create_function = """\
# function(cluster) {
#     return L.divIcon({
#     html: '<b>' + cluster.getChildCount() + '</b>',
#     className: 'marker-cluster marker-cluster-large',
#     iconSize: new L.Point(20, 20)
#     });
# }"""

# marker_cluster = MarkerCluster(
#     locations=locations, popups=popups
#     name='Practice Test',
#     overlay=True,
#     control=True,
#     icon_create_function=icon_create_function
# ).add_to(m)



# folium.LayerControl().add_to(m)

# #popups = ['{}<br>Backer Count = {}'.format(map_data.name, map_data.backers_count) for project in map_data.name]

# # folium.Marker(
# #     location=[40.67, -73.94], 
# #     popup='Add popup text here.',
# #     icon=folium.Icon(color='green', icon='ok-sign'),
# # ).add_to(marker_cluster)

# # folium.Marker(
# #     location=[44.67, -73.94],
# #     popup='Add popup text here.',
# #     icon=folium.Icon(color='red', icon='remove-sign'),
# # ).add_to(marker_cluster)

# # folium.Marker(
# #     location=[44.67, -71.94],
# #     popup='Add popup text here.',
# #     icon=None,
# # ).add_to(marker_cluster)

# #m.save(os.path.join('results', '1000_MarkerCluster0.html'))

# m

In [89]:
#%%time
m = folium.Map(location=[map_data.lat.mean(), map_data.lng.mean()], zoom_start=5)

marker_cluster = MarkerCluster(
    name='US Kickstarter Campaigns',
    overlay=True,
    control=True,
    icon_create_function=None
)

for k in range(1000):
    location = locations[k][0], locations[k][1]
    marker = folium.Marker(location=location)
    popup = popups[k]
    folium.Popup(popup, max_width='150%').add_to(marker)
    if list(df.state)[k] == 'live':
        icons = folium.Icon(color='lightgreen', icon='bullhorn').add_to(marker)
    elif list(df.state)[k] == 'successful':
        icons = folium.Icon(color='green', icon='ok-sign').add_to(marker)
    else:
        icons = folium.Icon(color='red', icon='remove-sign').add_to(marker)
    marker_cluster.add_child(marker)

marker_cluster.add_to(m)

folium.LayerControl().add_to(m)

m
#m.save(os.path.join('../../results', 'USfastClusterTEST.html'))

## Continue...

In [43]:
df.state.value_counts()

successful    67083
failed        37435
live          19267
Name: state, dtype: int64

#### Testing

In [ ]:
# for k in range(20):
#     if list(df.state)[k] == 'live':
#         icons = folium.Icon(color='blue', icon='ok-sign').add_to(marker_cluster)
#         print(f'The icon will be blue')
#     elif list(df.state)[k] == 'successful':
#         icons = folium.Icon(color='green', icon='remove-sign').add_to(marker_cluster)
#         print('The icon will be green')
#     else:
#         icons = folium.Icon(color='red', icon='ok-sign').add_to(marker_cluster)
#         print('This one failed')